# Predicting Machine Failure Type:

In [63]:
#Import Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve,classification_report, precision_recall_curve, auc, accuracy_score, precision_score, recall_score, f1_score
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings('ignore')

In [59]:
# Load the preprocessed data
data = pd.read_csv('Preprocessed_Data.csv')

# Rename columns to remove special characters
data.rename(columns={
    'Air temperature [K]': 'Air_temperature_K',
    'Process temperature [K]': 'Process_temperature_K',
    'Rotational speed [rpm]': 'Rotational_speed_rpm',
    'Torque [Nm]': 'Torque_Nm',
    'Tool wear [min]': 'Tool_wear_min'
}, inplace=True)

# Create the 'No failure' column
data['No failure'] = 1 - data['Machine failure']

# Define features and target
X = data[['Type', 'Air_temperature_K', 'Process_temperature_K', 'Rotational_speed_rpm', 'Torque_Nm', 'Tool_wear_min']]
y = data[['No failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']].idxmax(axis=1)

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [60]:
# Initialize stratified split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(X, y_encoded):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

# Apply SMOTE to oversample the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Define all possible target names
all_classes = label_encoder.classes_

In [61]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(random_state=42, max_iter=300),
    'AdaBoost': AdaBoostClassifier(random_state=42)
}

# Train models
trained_models = {}
y_preds = {}
y_probas = {}

for name, model in models.items():
    model.fit(X_train_res, y_train_res)
    y_preds[name] = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_probas[name] = model.predict_proba(X_test)
    else:
        # For SVM with probability=False
        y_probas[name] = model.decision_function(X_test)
        y_probas[name] = (y_probas[name] - y_probas[name].min()) / (y_probas[name].max() - y_probas[name].min())
    trained_models[name] = model

In [64]:
# Plot functions
def plot_confusion_matrix(y_test, y_pred, model_name):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=all_classes, yticklabels=all_classes)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.show()

def plot_roc_curve(y_test, y_proba, model_name):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    plt.figure(figsize=(10, 7))
    
    for i in range(len(all_classes)):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], lw=2, label=f'Class {all_classes[i]} (area = {roc_auc[i]:0.2f})')
    
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic - {model_name}')
    plt.legend(loc="lower right")
    plt.show()

def plot_precision_recall_curve(y_test, y_proba, model_name):
    precision = dict()
    recall = dict()
    pr_auc = dict()
    plt.figure(figsize=(10, 7))
    
    for i in range(len(all_classes)):
        precision[i], recall[i], _ = precision_recall_curve(y_test == i, y_proba[:, i])
        pr_auc[i] = auc(recall[i], precision[i])
        plt.plot(recall[i], precision[i], lw=2, label=f'Class {all_classes[i]} (area = {pr_auc[i]:0.2f})')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - {model_name}')
    plt.legend(loc="lower left")
    plt.show()

def plot_metrics(y_test, y_pred, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    metrics = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}
    plt.figure(figsize=(10, 7))
    bars = plt.bar(metrics.keys(), metrics.values(), color='blue')
    plt.xlabel('Metrics')
    plt.ylabel('Scores')
    plt.ylim([0, 1])
    plt.title(f'Metrics - {model_name}')
    
    # Annotate bars with the metric values
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), va='bottom')  # va: vertical alignment
    
    plt.show()

# Dropdown Menu Function
def display_plots(model_name, plot_type):
    y_pred = y_preds[model_name]
    y_proba = y_probas[model_name]
    
    if plot_type == 'Confusion Matrix':
        plot_confusion_matrix(y_test, y_pred, model_name)
    elif plot_type == 'ROC Curve':
        plot_roc_curve(y_test, y_proba, model_name)
    elif plot_type == 'Precision-Recall Curve':
        plot_precision_recall_curve(y_test, y_proba, model_name)
    elif plot_type == 'Metrics':
        plot_metrics(y_test, y_pred, model_name)

# Create Dropdown Widgets
model_dropdown = widgets.Dropdown(options=list(models.keys()), description='Model:')
plot_dropdown = widgets.Dropdown(options=['Confusion Matrix', 'ROC Curve', 'Precision-Recall Curve', 'Metrics'], description='Plot Type:')

# Display Interactive Widgets
interact(display_plots, model_name=model_dropdown, plot_type=plot_dropdown)

interactive(children=(Dropdown(description='Model:', options=('Logistic Regression', 'Random Forest', 'XGBoost…

<function __main__.display_plots(model_name, plot_type)>